# Parte 1. Sistemas multiagentes

## 1. Representación del entorno

El entorno se representa como una retícula rectangular de N x M casillas. En esta retícula hay diferentes categorías que se le asignan a los cuadros:

- Calle donde pueden transitar autos.
- Auto.
- Semáforo.

## 2. Percepciones del entorno

En el modelo los conductores deben tener percepciones del entorno de manera que les permita tomar decisiones.

Las situaciones pueden ser las siguientes:
- Tienen la calle frente a ellos libre.
- Tienen un auto frente a ellos.
- Tienen un semáforo frente a ellos (puede estar en color verde, amarillo o rojo).

## 3. Definir las acciones de los conductores

Para cada situación descrita anteriormente se definen acciones que pueden realizar los conductores.

### La calle enfrente está libre
